<a href="https://colab.research.google.com/github/ethansimrm/medical-mt/blob/main/T5_Translation_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install required dependencies
!pip install transformers==4.28.0 datasets evaluate sacrebleu torch #git+https://github.com/huggingface/accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
#Load in EN-FR subset of OPUS books
from datasets import load_dataset, load_dataset_builder
books = load_dataset("opus_books", "en-fr")

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
#ds_builder = load_dataset_builder("rotten_tomatoes")
#ds_builder = load_dataset_builder("wmt14", 'fr-en')
#ds_builder.info.description
#ds_builder.info.features

In [ ]:
#Entirety of OPUS books is a training dataset, so split into train:test with 80:20.
books = books["train"].train_test_split(test_size=0.2)

In [ ]:
#Inspect data; the split is random.
books["train"][0]

{'id': '67130',
 'translation': {'en': 'Since he had grown to manhood they no longer said in so many words:"Look at Jean and follow his example," but every time he heard them say"Jean did this--Jean does that," he understood their meaning and thehint the words conveyed.',
  'fr': "Depuis qu'il était homme, on ne lui disait plus: «Regarde Jean etimite-le!» mais chaque fois qu'il entendait répéter: «Jean a fait ceci,Jean a fait cela,» il comprenait bien le sens et l'allusion cachés sousces paroles."}}

In [38]:
books["train"].push_to_hub("OpusTrain")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/102 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [39]:
books["test"].push_to_hub("OpusTest")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/26 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
#Tokenise data - we must load the correct tokeniser for our model before we input parameters. This is based on SentencePiece.
from transformers import AutoTokenizer
checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
#NB: As long as data looks like {"input" : "XXXX" , "target" : "YYYY"}, it can be processed.

In [ ]:
#Specify a preprocessing function which allows us to tokenise source and target languages correctly AND prime T5 with the correct prompt before each sentence
source_lang = "en"
target_lang = "fr"
prefix = "translate English to French: "


def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]] 
    #We are essentially querying the dictionary here, whereby books["train"]["translation"][0]["en"] references the first English sentence above.
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True) #We will truncate sentences longer than 128 words long
    
    return model_inputs

#books["train"]["translation"][0]["en"] will give us the first English sentence

In [ ]:
#Apply this function over the training dataset over multiple elements simultaneously using the batched = True argument.
tokenized_books = books.map(preprocess_function, batched=True) 

Map:   0%|          | 0/101668 [00:00<?, ? examples/s]

Map:   0%|          | 0/25417 [00:00<?, ? examples/s]

In [ ]:
#Create a batch of examples and dynamically pad to hit length of longest sentence per batch
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [ ]:
#Load evaluation method
import evaluate
metric = evaluate.load("sacrebleu")

In [ ]:
import numpy as np

def postprocess_text(preds, labels): 
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True) #Convert back into words

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id) #Ignore padded labels added by the data collator to the test set
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels) #Remove leading and trailing spaces

    result = metric.compute(predictions=decoded_preds, references=decoded_labels) #BLEU score for provided input and references
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens) #Compute mean prediction length
    result = {k: round(v, 4) for k, v in result.items()} #Round score to 4dp
    return result

In [ ]:
#Ready to download model
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, GenerationConfig
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint) #Model is 242MB in size

In [25]:
training_args = Seq2SeqTrainingArguments( #Collects hyperparameters
    output_dir="test_t5_small_example_3",
    evaluation_strategy="epoch", #Evaluates at the end of each epoch
    learning_rate=2e-5, #Initial learning rate for AdamW
    per_device_train_batch_size=16, #Minibatch learning
    per_device_eval_batch_size=16, #Batch size for evaluation
    weight_decay=0.01, #Weight decay for loss computation; Loss = Loss + WD * sum (weights squared)
    save_total_limit=3, #Number of checkpoints to save
    num_train_epochs=2,
    predict_with_generate=True, #Use with ROUGE/BLEU and other translation metrics (see below)
    fp16=True, #Remove fp16 = True if not using CUDA
    push_to_hub=True,
)

trainer = Seq2SeqTrainer( #Saves us from writing our own training loops
    model=model,
    args=training_args,
    train_dataset=tokenized_books["train"],
    eval_dataset=tokenized_books["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

#However, these metrics require that we generate some text with the model rather than a single forward pass as with e.g. classification. 
#The Seq2SeqTrainer allows for the use of the generate method when setting predict_with_generate=True which will generate text for each sample in the evaluation set. 
#That means we evaluate generated text within the compute_metric function. We just need to decode the predictions and labels first.

/content/test_t5_small_example_3 is already a clone of https://huggingface.co/ethansimrm/test_t5_small_example_3. Make sure you pull the latest changes with `repo.git_pull()`.


In [26]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,1.706100,1.545824,6.119400,17.576900
2,1.716800,1.533271,6.209900,17.569200


TrainOutput(global_step=12710, training_loss=1.7192709285757093, metrics={'train_runtime': 3293.8135, 'train_samples_per_second': 61.733, 'train_steps_per_second': 3.859, 'total_flos': 5017466336575488.0, 'train_loss': 1.7192709285757093, 'epoch': 2.0})

In [27]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/231M [00:00<?, ?B/s]

Upload file runs/May25_20-50-55_8556331bf5f9/events.out.tfevents.1685047860.8556331bf5f9.270.4:   0%|         …

To https://huggingface.co/ethansimrm/test_t5_small_example_3
   9cb65ee..4c27662  main -> main

   9cb65ee..4c27662  main -> main

To https://huggingface.co/ethansimrm/test_t5_small_example_3
   4c27662..118abb1  main -> main

   4c27662..118abb1  main -> main



'https://huggingface.co/ethansimrm/test_t5_small_example_3/commit/4c27662bb7628237e30ec083ddee88689bf777c5'

In [32]:
text = "translate English to French: Legumes share resources with nitrogen-fixing bacteria."

In [33]:
from transformers import pipeline

translator = pipeline("translation", model="test_t5_small_example_3")
translator(text)

[{'translation_text': 'Legumes partagent des ressources avec des bactéries fixantes de nitrogen.'}]